In [ ]:
import os
from IPython.display import clear_output


notebook_dir = "/home/balabaevvl/courses/project/FITS/src/"
os.chdir(notebook_dir)

GPUs = [
    "GPU-e83bd31b-fcb9-b8de-f617-2d717619413b",
    "GPU-5a9b7750-9f85-49a5-3aae-fe07b1b7661d",
    "GPU-fe2d8dfd-06f2-a5c4-a7fd-4a5f23947005",
    "GPU-0c320096-21ee-4060-8731-826ca2febfab",
    "GPU-baef952c-6609-aace-3b78-e4e07788d5de",
    "GPU-3979d65b-c238-4e9c-0c1c-1aa3f05c56a1",
    "GPU-6c76a2c5-5375-aa06-11d4-0fddfac30e91",
]
os.environ["CUDA_VISIBLE_DEVICES"] = f"{GPUs[1]}"


In [ ]:
from fits.data.download import DownloadDatasetAirQuality

DownloadDatasetAirQuality()


In [ ]:
import pandas as pd
from fits.config import DatasetsPaths

df = pd.read_csv(
    DatasetsPaths.pm25.value,
    index_col="datetime",
    parse_dates=True,
)


In [ ]:
from fits.data.dataset import ModelMode, DatasetAirQuality

dataset = DatasetAirQuality(ModelMode.train)

for sample in dataset:
    break

sample


In [ ]:
from fits.modelling.framework import Train, Evaluate
from fits.data.dataset import DatasetAirQuality
from fits.modelling.CSDI.adapter import CSDIAdapter
from fits.data.dataloader import ForecastingDataLoader


train_loader, valid_loader, test_loader = ForecastingDataLoader(
    DatasetAirQuality, batch_size=128
)

csdi = CSDIAdapter().to("cuda")


In [ ]:
Train(csdi, train_loader, valid_loader)


In [ ]:
Evaluate(csdi, test_loader, ???normaliztion_stats???)


In [ ]:
# TODO: refactor:
"""
import pickle
import matplotlib.pyplot as plt


def plot_pm25(generated_outputs_nsample_path: str):
    def get_quantile(samples,q,dim=1):
        return torch.quantile(samples,q,dim=dim).cpu().numpy()

    path = generated_outputs_nsample_path
    with open(path, 'rb') as f:
        samples,all_target,all_evalpoint,all_observed,all_observed_time,scaler,mean_scaler = pickle.load( f)

    all_target_np = all_target.cpu().numpy()
    all_evalpoint_np = all_evalpoint.cpu().numpy()
    all_observed_np = all_observed.cpu().numpy()
    all_given_np = all_observed_np - all_evalpoint_np

    K = samples.shape[-1] #feature
    L = samples.shape[-2] #time length

    dataset = "airquality"
    if dataset == 'airquality':
        path = 'data/pm25/pm25_meanstd.pk'

        with open(path, 'rb') as f:
            train_mean, train_std = pickle.load(f)

        train_std_cuda = torch.from_numpy(train_std).cuda()
        train_mean_cuda = torch.from_numpy(train_mean).cuda()
        all_target_np=(all_target_np*train_std+train_mean)
        samples=(samples*train_std_cuda+train_mean_cuda)


    qlist =[0.05,0.25,0.5,0.75,0.95]
    quantiles_imp= []

    for q in qlist:
        quantiles_imp.append(get_quantile(samples, q, dim=1)*(1-all_given_np) + all_target_np * all_given_np)


    ###airquality###
    dataind = 10 #change to visualize a different sample

    plt.rcParams["font.size"] = 16
    fig, axes = plt.subplots(nrows=9, ncols=4,figsize=(24.0, 36.0))
    fig.delaxes(axes[-1][-1])

    for k in range(K):
        df = pd.DataFrame({"x":np.arange(0,L), "val":all_target_np[dataind,:,k], "y":all_evalpoint_np[dataind,:,k]})
        df = df[df.y != 0]
        df2 = pd.DataFrame({"x":np.arange(0,L), "val":all_target_np[dataind,:,k], "y":all_given_np[dataind,:,k]})
        df2 = df2[df2.y != 0]
        row = k // 4
        col = k % 4
        axes[row][col].plot(range(0,L), quantiles_imp[2][dataind,:,k], color = 'g',linestyle='solid',label='CSDI')
        axes[row][col].fill_between(range(0,L), quantiles_imp[0][dataind,:,k],quantiles_imp[4][dataind,:,k],
                        color='g', alpha=0.3)
        axes[row][col].plot(df.x,df.val, color = 'b',marker = 'o', linestyle='None')
        axes[row][col].plot(df2.x,df2.val, color = 'r',marker = 'x', linestyle='None')

        if col == 0:
            plt.setp(axes[row, 0], ylabel='value')
        if row == -1:
            plt.setp(axes[-1, col], xlabel='time')

    return fig
"""